## Deep Learning Methods

In [1]:
#Imports
# -*- coding: utf-8 -*-

import pandas as pd
import csv
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [3]:
dataDL_section = pd.read_csv('../TFM/Machine_Learning_Data/deep/datos_DL.csv',sep=';',engine='python')
dataDL_section.head(5)

,timestamp,dias,id,tipo_estado,estado
0,1420071301,4,101,1,140
1,1420071301,4,103,1,35
2,1420071301,4,105,1,173
3,1420071301,4,107,1,120
4,1420071301,4,108,1,143


In [4]:
#Comprobamos Dimensión
dataDL_section.shape

(11729306, 5)

In [5]:
dataDL_section['estado'].describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
estado,11729306.0,809.073745,1093.543012,1.0,98.0,345.0,1059.0,17982.0


**1.- Neural Network**

In [6]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras

from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer

In [7]:
print('Versión TensorFlow: ',tf.__version__)

Versión TensorFlow:  2.3.0


In [8]:
dataNN = dataDL_section[['timestamp','dias', 'id','estado','tipo_estado']].copy()
dataNN.head(5)

,timestamp,dias,id,estado,tipo_estado
0,1420071301,4,101,140,1
1,1420071301,4,103,35,1
2,1420071301,4,105,173,1
3,1420071301,4,107,120,1
4,1420071301,4,108,143,1


In [9]:
dataNN.dtypes

timestamp      int64
dias           int64
id             int64
estado         int64
tipo_estado    int64
dtype: object

In [10]:
X = dataNN[['timestamp','dias','id','estado']] # Features
#X = dataNN[['timestamp','id','estado']]

#Normalize data
#preprocesador = make_column_transformer((['timestamp','dias','id','media'], OneHotEncoder()))
#X = preprocesador.fit_transform(X)


y = dataNN['tipo_estado'] # Target variable

In [11]:
#División Train y Test
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=1)

In [12]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8796979, 4) (2932327, 4) (8796979,) (2932327,)


In [13]:
print('Entrenamiento: ',X_train.shape)
print('Test: ', X_test.shape)

Entrenamiento:  (8796979, 4)
Test:  (2932327, 4)


In [14]:
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [15]:
#determine the number of input features

model = keras.Sequential()
model.add(Dense(12, activation='relu', kernel_initializer='he_normal',input_dim=4)) #capa oculta
model.add( Dropout(0.5) ) #avoid overfitting

model.add(Dense(8,activation='relu', kernel_initializer='he_normal')) #capa oculta
model.add( Dropout(0.5) )

model.add(Dense(1,activation='sigmoid')) #capa salida
model.add( Dropout(0.5) )

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                60        
_________________________________________________________________
dropout (Dropout)            (None, 12)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1)                 0         
Total params: 173
Trainable params: 173
Non-trainable params: 0
__________________________________________________________

In [17]:
history = model.fit(X_train, y_train, batch_size=100, epochs=20, verbose=1, shuffle=True,validation_data=(X_test, y_test)) #58.55

#history = model.fit(X_train, y_train, batch_size=100, epochs=30, verbose=1, shuffle=True,\
                    #validation_data=(X_test, y_test)) #58.55
history

Epoch 1/20
87970/87970 [==============================] - 80s 915us/step - loss: 7.7873 - accuracy: 0.2925 - val_loss: -3.4918 - val_accuracy: 0.5855
Epoch 2/20
87970/87970 [==============================] - 82s 927us/step - loss: 7.7627 - accuracy: 0.2927 - val_loss: -3.4918 - val_accuracy: 0.5855
Epoch 3/20
87970/87970 [==============================] - 82s 929us/step - loss: 7.7618 - accuracy: 0.2930 - val_loss: -3.4918 - val_accuracy: 0.5855
Epoch 4/20
87970/87970 [==============================] - 81s 915us/step - loss: 7.7804 - accuracy: 0.2929 - val_loss: -3.4918 - val_accuracy: 0.5855
Epoch 5/20
87970/87970 [==============================] - 80s 908us/step - loss: 7.7757 - accuracy: 0.2929 - val_loss: -3.5106 - val_accuracy: 0.5855
Epoch 6/20
87970/87970 [==============================] - 81s 923us/step - loss: 7.7543 - accuracy: 0.2928 - val_loss: -3.5106 - val_accuracy: 0.5855
Epoch 7/20
87970/87970 [==============================] - 81s 924us/step - loss: 7.7877 - accuracy: 

In [18]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test,verbose=0)
print('Accuracy: %.2f' % (acc*100))

Accuracy: 58.55


In [21]:
predictions = model.predict(X_test)
print(predictions)
#df_predNN = pd.DataFrame({'Actual': y_test, 'Predicción': predictions})
#df_predNN.head(5)

[[0.9990119 ]
 [0.77611005]
 [0.99637955]
 ...
 [0.9675107 ]
 [0.77697325]
 [0.94267917]]


In [23]:
#Comprobando Predicciones
date_pred = '2021-01-01 10:00:00'
date_p = datetime.strptime(date_pred, "%Y-%m-%d %H:%M:%S")
timestamp = datetime.timestamp(date_p)
print(int(timestamp))


convert_date_time = int(timestamp)
day_pred = 5
id_pred = 10127
estado = 3000
abs_pred = ([[convert_date_time,day_pred,id_pred,estado]])
new_pred = model.predict(abs_pred)
print('Prediction: ',int(new_pred))

1609491600
Prediction:  1
